### Load Python Pakages

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#basics
import numpy as np
import pandas as pd 
import seaborn as sns
import time
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore")

#preprocessing

#feature engineering
from sklearn.feature_selection import mutual_info_classif


#algorithms
from xgboost import XGBClassifier
import xgboost as xgb

#model evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.metrics import RocCurveDisplay, confusion_matrix
from sklearn.metrics import roc_curve, confusion_matrix, ConfusionMatrixDisplay

#!pip3 install yellowbrick
from yellowbrick.features import FeatureImportances
from yellowbrick.classifier import ConfusionMatrix, ClassificationReport, ROCAUC, DiscriminationThreshold

#!pip3 install shap
import shap

from ipywidgets import interact, FloatSlider

random_state = 42

In [ ]:
#preprocessing
from sklearn.preprocessing import StandardScaler, PowerTransformer, MinMaxScaler, LabelEncoder,OneHotEncoder, OrdinalEncoder

#feature engineering
from sklearn.feature_selection import mutual_info_classif


#transformers and pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.base import clone
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn import set_config

#algorithms
from xgboost import XGBClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.base import clone
from lightgbm import early_stopping
from lightgbm import log_evaluation
from sklearn.linear_model import LogisticRegression


#model evaluation
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, log_loss, auc, accuracy_score, balanced_accuracy_score
from sklearn.metrics import make_scorer, RocCurveDisplay, confusion_matrix

# Optuna and visualization tools
import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

### Lets look into data

In [ ]:
# Read the data
train_df = pd.read_csv('/kaggle/input/playground-series-s4e7/train.csv', index_col=[0])
test_df = pd.read_csv('/kaggle/input/playground-series-s4e7/test.csv', index_col=[0])
#original_df = pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/train.csv', index_col=[0])

train_df.head()

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
#This function reduces memory usage. Reference: https://www.kaggle.com/code/jmascacibar/optimizing-memory-usage-with-insurance-cross-sell/notebook
def shrink_and_dummify(df):
    df["Vehicle_Age"] = df["Vehicle_Age"].apply(lambda x: 0 if x == '< 1 Year' else (1 if x == '1-2 Year' else 2)).astype('int8')
    df['Gender'] = df['Gender'].apply(lambda x: 0 if x == 'Female' else 1).astype('int8')
    df['Vehicle_Damage'] = df['Vehicle_Damage'].apply(lambda x: 0 if x == 'No' else 1).astype('int8')
    df['Age'] = df['Age'].astype('int8')
    df['Driving_License'] = df['Driving_License'].astype('int8')
    df['Region_Code'] = df['Region_Code'].astype('int8')
    df['Previously_Insured'] = df['Previously_Insured'].astype('int8')
    df['Annual_Premium'] = df['Annual_Premium'].astype('int32')
    df['Policy_Sales_Channel'] = df['Policy_Sales_Channel'].astype('int16')
    df['Vintage'] = df['Vintage'].astype('int16')
    
    if 'Response' in df.columns:
        df['Response'] = df['Response'].astype('int8')
    
    return df

In [ ]:
train_df = shrink_and_dummify(train_df)
test_df = shrink_and_dummify(test_df)

### Descpriptive statistics

In [ ]:
train_df.describe().T

In [ ]:
plt.figure(figsize=(10, 10))
palette_color = sns.color_palette('pastel')
explode = [0.05 for _ in range(train_df['Response'].nunique())]

# Plotting
train_df.groupby('Response')['Response'].count().plot.pie(
    colors=palette_color,
    explode=explode,
    autopct="%1.1f%%",
    shadow=True,  # Adding shadow for better visibility
    startangle=140,  # Start angle for better alignment
    textprops={'fontsize': 14},  # Adjust text size
    wedgeprops={'edgecolor': 'black', 'linewidth': 1.5}  # Adding edge color and width
)

# Adding a title
plt.title('Target Distribution', fontsize=18, weight='bold')

# Equal aspect ratio ensures that pie is drawn as a circle.
plt.axis('equal')

# Displaying the plot
plt.show()

### Grouping features for preprocessing purposes

In [ ]:
train_df.nunique().sort_values()

In [ ]:
train_df.info()

In [ ]:
# Store numerical and categorical features to different lists for visualization purposes
feature_list = [feature for feature in train_df.columns if not feature  == "Response"]

target = "Response"

binary_features = ['Previously_Insured','Driving_License', 'Gender', 'Vehicle_Damage', 'Vehicle_Age']

continuous_features = list(set(feature_list) - set(binary_features))

assert feature_list.sort() == (continuous_features + binary_features).sort()

### Exploratory Data Analysis

In [ ]:
#Data is huge just take some sample for eda
eda_df = train_df.sample(frac=0.01)

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(30, 20))
for var, subplot in zip(continuous_features, ax.flatten()):
    sns.boxplot(x='Response', y=var, data=eda_df, ax=subplot, palette='Set3')

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(30, 20))
for var, subplot in zip(binary_features, ax.flatten()):
    sns.barplot(x= var, y= 'Response', data=eda_df, ax=subplot, palette='Set3')

### Feature Engineering

In [ ]:
# determine the mutual information for features

mutual_df = eda_df[feature_list]
y_sampled = eda_df.Response
mutual_info = mutual_info_classif(mutual_df, y_sampled, random_state=random_state)

mutual_info = pd.Series(mutual_info)
mutual_info.index = mutual_df.columns
mutual_info = pd.DataFrame(mutual_info.sort_values(ascending=False), columns = ["Numerical_Feature_MI"] )
mutual_info.style.background_gradient("hot")

In [ ]:
#eda_df = pd.get_dummies(eda_df, columns=['Previously_Insured'])
#eda_df['Previously_Insured_Yes'] = eda_df['Previously_Insured_Yes'].astype('int')
#eda_df['Previously_Insured_No'] = eda_df['Previously_Insured_No'].astype('int')
#eda_df.head().T

In [ ]:
#eda_df = eda_df.rename(columns={'Vehicle_Age_1-2 Year': 'Vehicle_Age_1-2_Year', 'Vehicle_Age_< 1 Year': 'Vehicle_Age_less_than_1_Year',
#                                'Vehicle_Age_> 2 Years': 'Vehicle_Age_greater_than_2_Years'})
#eda_df.head().T

In [ ]:
#eda_df['Vehicle_Age'] = eda_df['Vehicle_Age'].apply(lambda x: '1-2_Year' if x == '1-2 Year' else ('less_than_1_Year' 
#                                                                  if x == '< 1 Year' else ('greater_than_2_Years')))
#eda_df.head().T

In [ ]:
#eda_df = pd.get_dummies(eda_df, columns=['Vehicle_Age'])
#eda_df['Vehicle_Age_less_than_1_Year'] = eda_df['Vehicle_Age_less_than_1_Year'].astype('int')
#eda_df['Vehicle_Age_1-2_Year'] = eda_df['Vehicle_Age_1-2_Year'].astype('int')
#eda_df['Vehicle_Age_greater_than_2_Years'] = eda_df['Vehicle_Age_greater_than_2_Years'].astype('int')
#eda_df.head().T

In [ ]:
#eda_df = pd.get_dummies(eda_df, columns=['Gender'])
#eda_df['Gender_Male'] = eda_df['Gender_Male'].astype('int')
#eda_df['Gender_Female'] = eda_df['Gender_Female'].astype('int')
#eda_df.head().T

In [ ]:
#eda_df = pd.get_dummies(eda_df, columns=['Vehicle_Damage'])
#eda_df['Vehicle_Damage_Yes'] = eda_df['Vehicle_Damage_Yes'].astype('int')
#eda_df['Vehicle_Damage_No'] = eda_df['Vehicle_Damage_No'].astype('int')
#eda_df.head().T

### Modeling and hyperparameter tuning

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
#A cross-validation function with early stopping. I used some crowdy code  to get the best iterations. If you have more elegant suggestions, please put the comment.

def cross_validate_score(model, data: pd.DataFrame, cv=None, test_data: pd.DataFrame = None, label: str = 'Response', include_original: bool = True, original_data: pd.DataFrame = None):
    if cv is None:
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    if test_data is None:
        raise ValueError("test_data must be provided")
    
    X = data.copy()
    y = X.pop(label)
    
    val_predictions = np.zeros(len(X))
    test_predictions = np.zeros(len(test_data))
    train_scores, val_scores = [], []
    
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train = X.iloc[train_idx].reset_index(drop=True)
        y_train = y.iloc[train_idx].reset_index(drop=True)
        X_val = X.iloc[val_idx].reset_index(drop=True)
        y_val = y.iloc[val_idx].reset_index(drop=True)
        
        if include_original:
            if original_data is None:
                raise ValueError("original_data must be provided when include_original is True")
            X_train = pd.concat([original_data.drop(label, axis=1), X_train]).reset_index(drop=True)
            y_train = pd.concat([original_data[label], y_train]).reset_index(drop=True)
        
        model_cloned = clone(model)
        
        if isinstance(model_cloned, XGBClassifier):
            eval_set = [(X_val, y_val)]
            model_cloned.fit(X_train, y_train, eval_set=eval_set, early_stopping_rounds=50, verbose=False)
            best_iteration = model_cloned.best_iteration
            train_preds_proba = model_cloned.predict_proba(X_train, iteration_range=(0, best_iteration))[:, 1]
            val_preds_proba = model_cloned.predict_proba(X_val, iteration_range=(0, best_iteration))[:, 1]
            test_preds_proba = model_cloned.predict_proba(test_data, iteration_range=(0, best_iteration))[:, 1]
        elif isinstance(model_cloned, LGBMClassifier):
            eval_set = [(X_val, y_val)]
            model_cloned.fit(X_train, y_train, eval_set=eval_set, eval_metric='auc', callbacks=[early_stopping(50)])
            best_iteration = model_cloned.best_iteration_
            train_preds_proba = model_cloned.predict_proba(X_train, num_iteration=best_iteration)[:, 1]
            val_preds_proba = model_cloned.predict_proba(X_val, num_iteration=best_iteration)[:, 1]
            test_preds_proba = model_cloned.predict_proba(test_data, num_iteration=best_iteration)[:, 1]
        elif isinstance(model_cloned, CatBoostClassifier):
            model_cloned.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=False)
            best_iteration = model_cloned.get_best_iteration()
            train_preds_proba = model_cloned.predict_proba(X_train)[:, 1]
            val_preds_proba = model_cloned.predict_proba(X_val)[:, 1]
            test_preds_proba = model_cloned.predict_proba(test_data)[:, 1]
        else:
            raise ValueError("Model type not supported for early stopping.")
        
        val_predictions[val_idx] = val_preds_proba
        train_scores.append(roc_auc_score(y_train, train_preds_proba))
        val_scores.append(roc_auc_score(y_val, val_preds_proba))
        
        print(f'Fold {fold}: {val_scores[-1]:.5f}')
        
        test_predictions += test_preds_proba / cv.get_n_splits()
        
    print(f'Val Score: {np.mean(val_scores):.7f} ± {np.std(val_scores):.7f} | Train Score: {np.mean(train_scores):.7f} ± {np.std(train_scores):.7f} | {label}')
    
    return val_scores, val_predictions, test_predictions

In [ ]:
cv_summary, oof_predictions, test_predictions = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

### XGBoost

In [ ]:
xgb_params = {
    
    'n_estimators': 10000,
    'eta': 0.05,
    'alpha':  0.2545607592482198,
    'subsample': 0.8388163485383147, 
    'colsample_bytree': 0.2732499701466825, 
    'max_depth': 16,
    'min_child_weight': 5,
    'gamma': 0.0017688666476104672,
    'eval_metric': 'auc',
    'max_bin': 262143, #a weird max_bin, for reference: https://www.kaggle.com/competitions/playground-series-s4e7/discussion/516265   
    'tree_method': 'gpu_hist',
}

xgb_tuned = XGBClassifier(**xgb_params, random_state=random_state)

In [ ]:
%%time
cv_summary['xgb'], oof_predictions['xgb'], test_predictions['xgb'] = cross_validate_score(xgb_tuned, data=train_df, test_data=test_df, include_original=False)

### CatBoost

In [ ]:
catb_params = {
    'iterations': 10000,
    'eval_metric': 'AUC',
    'task_type': 'GPU',
    'learning_rate': 0.1,
    'depth': 9,
    'l2_leaf_reg': 55.37964307854247,
    'max_bin': 404,
    'bagging_temperature': 0.017138393608280057,
    'random_strength': 9.256288011643901
}


#defining all features as cateegorical makes huge impact 
#reference: https://www.kaggle.com/code/rohanrao/automl-grand-prix-1st-place-solution
catb_tuned = CatBoostClassifier(**catb_params, random_state=random_state, logging_level='Silent',cat_features=test_df.columns.values)

In [ ]:
%%time
cv_summary['catb'], oof_predictions['catb'], test_predictions['catb'] = cross_validate_score(catb_tuned, data=train_df, test_data=test_df, include_original=False)

### Neural Network

In [ ]:
def cross_validate_score_nn(model_builder, data: pd.DataFrame, test_data: pd.DataFrame, label: str = 'Response', cv=None):
    if cv is None:
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    X = data.copy()
    y = X.pop(label)
    
    val_predictions = np.zeros(len(X))
    test_predictions = np.zeros(len(test_data))
    val_scores = []
    
    input_shape = {feature: int(data[feature].max()) for feature in categorical_features}
    
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train = X.iloc[train_idx].reset_index(drop=True)
        y_train = y.iloc[train_idx].reset_index(drop=True)
        X_val = X.iloc[val_idx].reset_index(drop=True)
        y_val = y.iloc[val_idx].reset_index(drop=True)
        
        model = model_builder(input_shape)
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                      loss='binary_crossentropy',
                      metrics=[keras.metrics.AUC(name='auc')])
        
        X_train_inputs = {feature: X_train[feature].values for feature in categorical_features}
        X_train_inputs['numerical'] = X_train[numerical_features].values
        
        X_val_inputs = {feature: X_val[feature].values for feature in categorical_features}
        X_val_inputs['numerical'] = X_val[numerical_features].values
        
        model.fit(X_train_inputs, y_train, epochs=4, batch_size=1024, validation_data=(X_val_inputs, y_val), verbose=0)

        val_preds_proba = model.predict(X_val_inputs).flatten()
        val_predictions[val_idx] = val_preds_proba
        val_scores.append(roc_auc_score(y_val, val_preds_proba))
        
        # Predict on test data
        test_inputs = {feature: test_data[feature].values for feature in categorical_features}
        test_inputs['numerical'] = test_data[numerical_features].values
        test_preds_proba = model.predict(test_inputs).flatten()
        test_predictions += test_preds_proba / cv.get_n_splits()
        
        print(f'Fold {fold}: {val_scores[-1]:.5f}')
    
    print(f'Val Score: {np.mean(val_scores):.7f} ± {np.std(val_scores):.7f} | {label}')
    
    return val_scores, val_predictions, test_predictions

In [ ]:
#Data curation to deal with high cardinality with embedding layers.
def prepare_data(train_df, test_df, numerical_features, categorical_features):
    # Concatenate train and test data for consistent encoding
    combined_data = pd.concat([train_df, test_df], keys=['train', 'test']).reset_index(level=0)
    
    # Ordinal encode categorical features
    ordinal_encoder = OrdinalEncoder()
    combined_data[categorical_features] = ordinal_encoder.fit_transform(combined_data[categorical_features])
    
    # Split back into train and test sets
    train_data = combined_data[combined_data['level_0'] == 'train'].drop(columns=['level_0'])
    test_data = combined_data[combined_data['level_0'] == 'test'].drop(columns=['level_0'])
    
    return train_data, test_data

# Example usage
numerical_features = ['Driving_License', 'Previously_Insured']
categorical_features = ['Region_Code', 'Policy_Sales_Channel', 'Vintage', 'Annual_Premium', 'Age', 'Gender', 'Vehicle_Damage', 'Vehicle_Age']

train_data, test_data = prepare_data(train_df, test_df, numerical_features, categorical_features)

In [ ]:
#reference: https://www.kaggle.com/code/paddykb/ps-s4e7-keras-haz-insurance-losses/notebook
#NN with embedding layers to  deal with high cardinality
from tensorflow import keras
from tensorflow.keras import layers


def create_model(input_shape):
    inputs = []
    flat_embeddings = []

    for feature, input_dim in input_shape.items():
        output_dim = min(64, round(1.6 * (input_dim + 1) ** 0.56))  
        input_layer = keras.Input(shape=(1,), name=feature)
        embedding_layer = layers.Embedding(input_dim=input_dim + 1, output_dim=output_dim)(input_layer)  
        embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
        embedding_layer = layers.Flatten()(embedding_layer)
        inputs.append(input_layer)
        flat_embeddings.append(embedding_layer)
        
    numerical_input = keras.Input(shape=(len(numerical_features),), name='numerical')
    inputs.append(numerical_input)

    concatenated_inputs = layers.Concatenate()(flat_embeddings + [numerical_input])
    concatenated_inputs_bn = layers.BatchNormalization()(concatenated_inputs)

    x = layers.Dense(256, activation='mish')(concatenated_inputs_bn)
    x = layers.BatchNormalization()(x)
    x = layers.Concatenate()([x, concatenated_inputs_bn])
    x = layers.Dense(128, activation='mish')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)

    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
%%time
cv_summary['nn'], oof_predictions['nn'], test_predictions['nn'] = cross_validate_score_nn(create_model, train_data, test_data=test_data)

### Performance Summary

In [ ]:
#performance summary for base learners
transposed_df = cv_summary.transpose()
transposed_df.columns = ['fold1','fold2','fold3','fold4','fold5']
transposed_df['Mean'] = transposed_df.mean(axis=1)
transposed_df['Std'] = transposed_df.std(axis=1)
transposed_df.sort_values(by = 'Mean', ascending=False).style.background_gradient('Dark2_r')

In [ ]:
def use_models_from_previous_version(df1, df2):
    df2 = df2.rename(columns={'xgb': 'xgb_01'})
    df2 = df2.rename(columns={'catb': 'catb_01'})
    merged_df = pd.concat([df1.reset_index(drop=True), df2.reset_index(drop=True)], axis=1)
    merged_df
    return merged_df

oof_predictions = use_models_from_previous_version(oof_predictions, oof_preds_version1)
test_predictions = use_models_from_previous_version(test_predictions, test_preds_version1)

In [ ]:
sns.set(font_scale=1.2, style="whitegrid")
correlation_train = oof_predictions.corr()
mask = np.triu(np.ones_like(correlation_train, dtype=bool))
plt.figure(figsize=(20, 20))
sns.heatmap(correlation_train, 
            mask=mask, 
            annot=True, 
            fmt='.3f', 
            cmap='coolwarm', 
            square=True, 
            linewidths=.5, 
            cbar_kws={"shrink": .75})
plt.title('Model Diversity Check - Correlation Heatmap', fontsize=20, pad=20)
plt.show()

### Voting vs Stacking

In [ ]:
#probability averaging
def soft_voting_ensemble(oof_df, y_actual):

    mean_prob = oof_df.mean(axis=1)
    ensemble_auc = roc_auc_score(y_actual, mean_prob)
    print(f'Ensemble AUC: {ensemble_auc:.5f}')
    
    return mean_prob, ensemble_auc

In [ ]:
_ , ensemble_auc = soft_voting_ensemble( oof_predictions, train_df.Response)

In [ ]:
#parameters suggested for meta model                                                                                                 
meta_model_params = {
'n_estimators': 43, 'alpha': 0.000759453839369262, 'subsample': 0.8635904939859487, 'colsample_bytree': 0.7579443772400538,
    'max_depth': 7, 'min_child_weight': 5, 'learning_rate': 0.13688008280542863, 'gamma': 0.19965095682630274}

meta_model = XGBClassifier(**meta_model_params, random_state=random_state)

In [ ]:
#%%time

#Deciding which models to include ensemble
#from sklearn.feature_selection import RFECV

min_features_to_select = 1

# Create a pipeline with preprocessor, RFECV, and LGBMClassifier
#pipeline = Pipeline([
#    ('rfecv', RFECV(estimator=meta_model,
#                    step=1,
#                    cv=cv,
#                    scoring="roc_auc",
#                    min_features_to_select=min_features_to_select,
#                    n_jobs=-1,))
#])

# Fit the pipeline on the training data
#pipeline.fit(oof_predictions, train_df.Response)

#CV score
#print("Best CV score: ")
#selected_models = np.array( oof_predictions.columns)[pipeline.named_steps['rfecv'].support_]
#print( pipeline.named_steps['rfecv'].cv_results_["mean_test_score"][len(selected_models) - 1])



# Selected features after RFECV
#print('Number of evaluated models:', len(oof_predictions.columns))
#print('Number of selected models for ensemble:', len(selected_models))
#print("Selected models:", selected_models)

In [ ]:
#above code selects following models:
selected_models = ['xgb_01', 'catb_01', 'nn', 'catb']

### Submission

In [ ]:
meta_model = meta_model.fit(oof_predictions[selected_models], train_df.Response)

In [ ]:
preds_test =  meta_model.predict_proba(test_predictions[selected_models])[:, 1]

In [ ]:
output = pd.DataFrame({'id': test_df.index,
                       'Response': preds_test})

In [ ]:
#use data leakage
#reference: https://www.kaggle.com/competitions/playground-series-s4e7/discussion/520253
#I guess it can be written much more elegant. 
reversed_labels = test_df.join(original_df.set_index(list(test_df.columns)),on=list(test_df.columns),how='inner')
df_assign =  1 - reversed_labels['Response']
sub = output.set_index('id')
sub['id'] = sub.index
sub = sub[['id', 'Response']]
sub['Response'].loc[df_assign.index] = df_assign
sub.to_parquet('submission.parquet', index=False)
sub.head()

In [ ]:
oof_predictions.to_parquet('oof_predictions.parquet', index=False)
test_predictions.to_parquet('test_predictions.parquet', index=False)